In [42]:
import regex
import pandas as pd
from konlpy.tag import Okt

In [43]:
okt = Okt()

In [44]:
test = """군 당국은 23일 일본 해상자위대 초계기가 이날 남해 이어도 인근 해상에서 우리 해군 함정(대조영함)을 향해 
근접 위협비행을 했다며 일본 측을 강력 규탄했다.
서욱 합동참모본부 작전본부장(육군 중장)은 이날 국방부 청사에서 긴급 기자회견을 갖고 "오늘 오후 2시3분께 이어도 
인근 해상에서 일본 초계기가 우리 해군 함정을 명확하게 식별한 상황임에도 불구하고 거리 약 540ｍ, 고도 약 60~70ｍ 저고도로 근접 위협비행을 
한 것은 명백한 도발행위로 간주한다"고 밝혔다.
서 중장은 "작년 12월 20일 일본의 저고도 근접 위협비행과 관련해 그동안 우리 한국은 인내하면서 절제된 대응을 하였음에도 불구하고 일본은 올해 1월 18일, 
1월 22일에도 우리 해군 함정에 대해 근접 위협비행을 실시했다"고 말했다. 
그는 "이러한 사실에 대해 일본 정부에 분명하게 재발방지를 요청했음에도 
오늘 또다시 이런 저고도 근접위협비행을 한 것은 우방국 함정에 대한 명백한 도발행위이므로 일본의 저의를 의심하지 않을 수 없으며,
이를 강력하게 규탄한다"고 강조했다.
그는 "또 다시 이런 행위가 반복될 경우, 우리 군의 대응행동수칙에 따라 강력하게 대응해 나갈 것"이라고 덧붙였다. 
일본 초계기는 지난달 20일에도 조난한 북한 선박 구조에 나선 해군 광개토대왕함을 향해 저공으로 위협적인 비행을 해 논란이 된 바 있다.
당시 북한 어선 구조작전 중이던 광개토대왕함은 근접하는 일본 초계기를 향해 경고통신을 하지 않았지만, 
이날 경계작전 중이던 대조영함은 일본 초계기를 향해 '접근하지 말라'는 취지의 경고통신을 한 것으로 전해졌다. 
정경두 국방부 장관은 이날 오후 기자간담회 도중 일본 초계기가 이어도 근해에서 우리 해군 함정을 향해 근접 비행을 했다는 소식을 
듣고 상황 조치를 위해 급히 자리를 떴다.
정 장관은 기자간담회에서 일본이 정치적 의도를 가지고 해상초계기 사격통제 레이더 조사(照射·겨냥해서 비춤) 문제를 제기했다는 견해를 밝혔다. 
그는 한일 '레이더-저공위협비행' 갈등 관련 질문에 "일본은 아베 수상과 관방장관, 방위상, 외무상, 통합막료장(우리의 합참의장)
까지 군사, 외교와 관련된 모든 분들이 나와서 (레이더 조사 문제를) 언급을 했다"며 "그런 측면에서 정치적인 의도가 있는 것 아니겠냐"고 말했다. 
정 장관은 "이게 왜 정치적인 문제와 연결되냐면, 어제 러시아와 일본은 북방영토 협상을 했다. 러시아가 북방영토를 내놓겠다고 
얘기하지 않을 것이 뻔해 가져올 보따리가 없다. 지지율 면에서 유리할 것이 없다"며 "그런 부분까지 연계된 것인지는 모르겠지만, 
(레이더 조사 문제 제기는) 정치적 의도를 가지고 있었던 것은 분명하다"고 말했다.
그러면서 "이 건 관련해서 일본이 논리적으로나 국제법적으로 한국의 주장을 뛰어넘을 수는 없다"며 "그렇기 때문에 더는 이것을 가지고 
(한국과) 협의하지 않겠다고 출구 전략을 편 것 같다"고 말했다. """

In [45]:
def get_rel_words(test):
    stemmed_normalized = okt.morphs(test, norm=True, stem=True)
    part_of_sentence = okt.pos(test)
    df = pd.DataFrame(part_of_sentence, columns=['word', 'pos'])
    df['stemmed'] = stemmed_normalized
    relevant_word_types = ['Noun', 'Verb', 'Adjective', 'Adverb']
    df['rel'] = df.apply(lambda x: x.pos in relevant_word_types, axis=1)
    relevant_words = df[df.rel]
    return relevant_words

In [46]:
relevant_words = get_rel_words(test)

In [47]:
relevant_words = relevant_words.groupby(['stemmed', 'pos']).word.count().reset_index()

In [48]:
relevant_words.sort_values(by='word', ascending=False)
relevant_words = relevant_words.rename(columns={'word':'count', 'stemmed': 'word'})

In [49]:
import urllib3
from bs4 import BeautifulSoup

In [50]:
http = urllib3.PoolManager()
r = http.request('GET', 'https://en.wiktionary.org/wiki/Wiktionary:Frequency_lists/Korean_5800')
soup = BeautifulSoup(r.data, 'lxml')

In [51]:
data = []
table = soup.find('table')
table_body = table.find('tbody')

rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele])

In [52]:
dict_wf = pd.Series([regex.sub('\([\w]*\)', '', j) for x in [d.split('\n')[1:] for i in data for d in i] for j in x]).to_dict()
dict_wf = { (v, k) for k,v in dict_wf.items()}

In [53]:
frequency_list = pd.DataFrame(dict_wf, columns=['word', 'frequency'])

In [54]:
frequency_list.dtypes

word         object
frequency     int64
dtype: object

In [56]:
frequency_list.to_csv('frequency_list.csv')

In [72]:
relevant_words.dtypes

word     object
pos      object
count     int64
dtype: object

In [79]:
pd.merge(relevant_words, frequency_list, on='word', how='left').sort_values(by='frequency', ascending=False)

,word,pos,count,frequency
9,거리,Noun,1,5154.0
136,이,Noun,2,5117.0
189,해,Noun,1,5052.0
112,약,Noun,2,5013.0
182,편,Noun,1,4884.0
...,...,...,...,...
188,합참의장,Noun,1,NaN
191,해상,Noun,3,NaN
192,해상자위대,Noun,1,NaN
196,협상,Noun,1,NaN


In [37]:
f = open('nextflix_subtitles.txt')

text = f.read()

In [39]:
text = text.split('\n')

In [41]:
text[1:]

['[주제곡]',
 '[아이들이 시끌벅적하다] [매미 울음]',
 '[잔잔한 음악]',
 '[학교 종이 울린다]',
 '- (아이1) 안녕 - (아이2) 안녕, 내일 봐',
 '- (아이3) 아, 비 와! - (아이4) 아, 뭐야, 비 오잖아',
 '[하늘이 우르릉거린다]',
 '[의미심장한 효과음]',
 '[TV에서 소리가 흘러나온다]',
 '(TV 속 앵커1) 속보입니다',
 '경기도 진양시에서 하교 중이던',
 '[무거운 음악] 초등학생 어린이가 납치되는 사건이 발생해',
 '경찰이 수사에 나섰습니다',
 '[소란스럽다]',
 '(기자1) 윤정 양은 평소 어떤 친구였나요?',
 '- (기자2) 윤정 양과 아는 사이였나요? - (아이5) 네',
 '(기자2) 같은 반 친구였어요?',
 '(기자1) 김윤정 양 유괴 사건에 대해서 어떻게 생각하세요?',
 '(어린 해영) 형! 형!',
 '(TV 속 앵커2) 김윤정 어린이 유괴 사건의',
 '결정적인 용의자가 나타났습니다 [카메라 셔터음]',
 '협박 편지와 범죄 현장에서 발견된 지문 등으로 지목된',
 '용의자 서형준은 현재 상진대 의대에 재학 중인...',
 '[TV 뉴스가 계속된다] (어린 해영) 윤정이 데려간 사람 여자였는데',
 '[무거운 음악] [사이렌이 울린다]',
 '(형사1) 어, 어, 시간이 없다니까, 지금 소재 파악하고...',
 '(어린 해영) [흐느끼며] 아저씨, 우리 형 잘못한 거 아니야',
 '우리 형 살려 줘요!',
 '형아!',
 '(형사2) 카드 내역 뽑았나? [형사3이 말한다]',
 '(어린 해영) 아저씨',
 '아저씨, 아니에요, 남자 아니에요!',
 '(형사2) 뭐가 아니야?',
 '(어린 해영) 아저씨, 범인 남자 아니에요, 예?',
 '- (형사4) 집에 얼른 들어가 - (형사5) 카드사에서 연락 안 왔어?',
 '(형사6) 제가 통화했고 자료 다 넘겼어요',
 '(형사5) 윤정이네서는 아무 소식 없었고?',
 '(형사6) 아직요',
 '[흐느낀다]',
 '[애잔한 음악

In [25]:
import re

In [36]:
get_rel_words(re.sub('[^\w , .]', '', text[25:]))

ValueError: Length of values (7606) does not match length of index (7590)